# News

In [ ]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 
!pip install --upgrade google-auth-oauthlib
!pip install openai

In [4]:
# Imports and APIs
import fetch_videos
import openai
import os
import pandas as pd

# Ensure you have set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

## Channel Definitions and Setups

In [5]:
# Dictionary of channel IDs and their corresponding names
channels = {
    "CNBC_TV": "UCrp_UI8XtuYfpiqluWLD7Lw",
    "DeepWater": 'UCQCNLsdpDV1XSHH4V8WQuPA',
    "BloombergTechnology": 'UCrM7B7SL_g1edFOnmj-SDKg',
    "YahooFinance": 'UCEAZeUIeJs0IjQiqTCdVSIg'
}

# Function to get channel name from ID
def get_channel_name_by_id(channel_id, channels):
    for name, id in channels.items():
        if id == channel_id:
            return name
    return None

# Example usage
channel_id = channels["CNBC_TV"]  # Change this to the desired channel ID
channel_name = get_channel_name_by_id(channel_id, channels)
print(f"Channel ID: {channel_id}")
print(f"Channel Name: {channel_name}")

Channel ID: UCrM7B7SL_g1edFOnmj-SDKg
Channel Name: BloombergTechnology


## Get Today's CNBC Videos

In [6]:
summary_file_name_today = f'summaries_{channel_name}_{fetch_videos.get_formated_date_today()}'
summary_file_name_today_cvs = f'{summary_file_name_today}.csv'
summary_file_name_today_html = f'{summary_file_name_today}.html'

In [7]:
## Fetch videos
period_type = 'today'  # 'today', 'days', 'weeks', 'months'
number = 4  # The 'today' setting does not use 'number', adjust if using other settings
start_date, end_date = fetch_videos.get_date_range(period_type, number)
df_videos_today = fetch_videos.fetch_videos(start_date, end_date, channel_id,summary_file_name_today_cvs)

Appended 4 new videos to summaries_BloombergTechnology_2024-08-06.csv.


/Users/sliu/src/mysrc/investing/razorback_investing/news/fetch_videos.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, new_videos_df], ignore_index=True).sort_values(by=['Published At'])


In [8]:
## Filter Vidoes by time
# Assume df_videos is your initial DataFrame loaded with video data
hour_range1 = '0-8'
hour_range2 = '8-10'
hour_range3 = '10-12'
hour_range4 = '12-14'
hour_range5 = '14-16'
hour_range6 = '16-24'

# Filter videos from the last 3 days
filtered_df = fetch_videos.filter_videos_by_date_and_time(df_videos_today, 'today', 1)
# fetch_videos.display_df(filtered_df)

In [9]:
## Add transcripts
df_videos_with_transcripts = fetch_videos.add_transcripts_to_df(filtered_df)
# fetch_videos.display_df(df_videos_with_transcripts)

### AI Summary 

In [10]:
## Assign Tasks: Get Summary
task_summary = """I would like you to summarize the transcript with the following instructions: 
First categorize the video content.The category should be one of the following: Crypto, Macro, Politics, Technology Stocks, or Other. 
Then summarize the stocks that are mentioned in this video.
Then provide key takeaways in a bullet point format.
Finally, use one sentence to summarize the overall sentiment of the stocks mentioned in the video.
Please print the summary in a human-readable format like the following: 
Category: Technology Stocks
Stock mentioned: Nvidia, Tesla, Apple
Key takeaways:
* Nvidia reported record revenue growth.
* Tesla announced a new electric vehicle model.
* Apple released a new software update which will make its products more secure.
Sentiment: Cautiously Positive
"""

# Get summaries for all transcripts
df_summaries = fetch_videos.apply_tasks_on_all_transcripts(df_videos_with_transcripts, client, task_summary)

# Save summaries to a CSV file
df_summaries.to_csv(summary_file_name_today_cvs, index=False)

In [11]:
## Save summaries to an HTML file 
df_summaries = pd.read_csv(summary_file_name_today_cvs)
html_content = fetch_videos.get_html_content_summary_only(df_summaries)
# print(html_content)

# Save the HTML content to an HTML file
with open(summary_file_name_today_html, 'w') as file:
    file.write(html_content)

In [12]:
import fetch_videos
## Combine title and trascripts into a file
def save_titles_and_transcripts_to_txt(df, file_name):
    """
    Combines all titles and transcripts into a single string, saves it to a text file,
    and returns the combined string.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the video data.
    - file_name (str): The name of the text file to save the combined string.

    Returns:
    - str: The combined string of all titles and transcripts.
    """
    combined_text = ""
    for index, row in df.iterrows():
        title = row['Title']
        transcript = row['Transcript']
        combined_text += f"Title: {title}\nTranscript:\n{transcript}\n\n"

    with open(file_name, 'w') as file:
        file.write(combined_text)

    return combined_text

# save titles and transcripts to a text file
summary_file_name_today_txt = f'{summary_file_name_today}.txt'
save_titles_and_transcripts_to_txt(df_summaries, summary_file_name_today_txt)

"Title: Stock Turnaround and Google's Monopoly Ruling | Bloomberg Technology\nTranscript:\nthe story monday was about some of the\nmega-cap tech names, some of those that have fueled the gains in the first half\nof the year. some of those names you were\nparticularly concerned about. so you sat on the sidelines away from\nthem. how do you react to the last 24 hours\nthen? well, it's interesting that with that\nsector, it wasn't really a monday story per se.\nit had started july 10th. the nasdaq drawdown had reached 13 and a\nhalf percent yesterday. so there was already quite a bit of\nslide going on. and i think that does stem from the\novervaluation. it's pretty stunning when you see a name\nlike in video that had been down 25 to 30% and was still trading at about 59\ntimes earnings, 30 times sales after such a violent correction.\ni think even today when you look at apple still being negative with this\nkind of bounce back rally today. it does feel to me like a rotation of\nleadershi

In [13]:
import fetch_videos
task2 = """Please tell me anything discussed in the file about small cap stocks. 
Please also provide which video or speaker side this
"""
with open(summary_file_name_today_txt, 'r') as file:
    all_transcripts = file.read()
summary = fetch_videos.apply_task_in_chunks(all_transcripts, client, task2,chunk_size=10000)
print(summary)

Based on the transcripts provided, there are no specific discussions or mentions about small-cap stocks. The focus primarily lies on mega-cap technology stocks, the impact of their valuations, and investor strategies, as well as political developments and their potential influence on the tech sector.

The first transcript titled "Stock Turnaround and Google's Monopoly Ruling | Bloomberg Technology" primarily features a conversation with a speaker named David, where discussion revolves around mega-cap tech stocks, valuations, and earnings in the technology sector.

The second transcript titled "Big Tech Traders Are Not Buying the Dip" does not mention small caps but rather discusses political aspects involving vice-presidential picks and their implications on tech policy.

The third transcript titled "Harris Taps Minnesota’s Walz for Ticket" includes market coverage and focuses on technology stocks, including evaluations of their performance after a market dip.

Unfortunately, none of t

### AI for all transcripts

## Send emails

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
import os
import fetch_videos

email_user = os.getenv('EMAIL_USER')
email_password = os.getenv('EMAIL_PASSWORD')
email_send1 = 'sliu810@gmail.com'
email_send2 = 'zhengwang827@gmail.com'

recipients = [email_send1]
# Set up the MIME
message = MIMEMultipart()
message['From'] = email_user
message['To'] = email_send1
message['Subject'] = f'summaries_{channel_name}_{fetch_videos.get_formated_date_today()}'

# load df_summaries from disk and get the html content
df_summaries = pd.read_csv(summary_file_name_today_cvs)
html_content= fetch_videos.get_html_content_summary_only(df_summaries)

# Attach the HTML content to the email
message.attach(MIMEText(html_content, 'html'))

# Function to send email
def send_email():
    try:
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)  # Outlook SMTP server
        #server.set_debuglevel(1)  # Enable debugging output
        server.starttls()
        server.login(email_user, email_password)
        text = message.as_string()
        server.sendmail(email_user, recipients, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Send the email
# send_email()

## Random one off experiement 

In [ ]:
# print(transcript)

In [ ]:
# transcript = fetch_videos.get_transcript('GabBf791bdY')
# print(transcript)
# # Example task
# task2 = """I would like you to summarize the text which describes Tesla FSD.
# Please say in what senario FSD performed well and waht senario it didn't perform well.
# Format like this:
# Title: 
# Key takeaways:
# * FSD performed well in ...
# * FSD had some challenges in ....
# Improvement over pervious version if any.
# """
# task3 = """could you print the transcript with two person speaking into a human readible format and maintain the original content?"""

# summary = get_summary(transcript, client, task3)
# print(summary)

In [ ]:
def get_summary_in_chunks(transcript, client, task, chunk_size=500):
    """
    Generates summaries in chunks to avoid truncation.

    Parameters:
    - transcript (str): The full transcript text.
    - client (object): The client to use for generating summaries.
    - task (str): The task identifier for the summary generation.
    - chunk_size (int): The maximum size of each chunk.

    Returns:
    - str: Combined summary of all chunks.
    """
    import textwrap

    # Split the transcript into chunks
    chunks = textwrap.wrap(transcript, chunk_size)
    full_summary = ""

    for chunk in chunks:
        summary = get_summary(chunk, client, task)
        full_summary += summary + "\n"

    return full_summary

In [ ]:
transcript = fetch_videos.get_transcript('GUM2FSuqOoE')
# print(transcript)
# Example task
task2 = """I would like you to summarize the text which describes Tesla FSD.
Please say in what senario FSD performed well and waht senario it didn't perform well.
Format like this:
Title: 
Key takeaways:
* FSD performed well in ...
* FSD had some challenges in ....
Improvement over pervious version if any.
"""
task3 = """print the transcript in a more human reaible fashion"""

summary = get_summary(transcript, client, task3)
print(summary)